In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
spark = SparkSession.builder.appName("Ngram_filter").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [2]:
'''df = spark.read.csv(['../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-0.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-1.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-2.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-3.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-4.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-5.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-6.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-7.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-8.csv',
'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-9.csv'], sep="\t", header=False)'''

'df = spark.read.csv([\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-0.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-1.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-2.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-3.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-4.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-5.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-6.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-7.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-8.csv\',\n\'../data/eng-gb-all/1/googlebooks-eng-gb-all-1gram-20090715-9.csv\'], sep="\t", header=False)'

In [3]:
df = spark.read.csv(['../../data/eng-all/googlebooks-eng-all-1gram-20090715-0.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-14.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-1.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-15.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-2.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-1.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-3.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-2.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-4.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-3.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-6.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-4.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-7.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-6.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-8.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-7.csv',
                     '../../data/eng-all/googlebooks-eng-all-1gram-20090715-9.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-8.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-0.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-9.csv',
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-11.csv',  
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-12.csv', 
                     '../../data/eng-all/googlebooks-eng-all-2gram-20090715-13.csv',], sep="\t", header=False)


In [4]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [5]:
df.count()

1353072200

In [6]:
def is_alpha(x):
    return x.replace(" ","").isalpha()

is_alpha_udf = udf(is_alpha, BooleanType())

In [7]:
df = df.filter(is_alpha_udf(df['ngram']))

In [8]:
df = df.sort(['ngram', 'year'], ascending=True).cache()

In [9]:
df.count()

1069619590

In [11]:
df.coalesce(1).write.csv("eng-all-only-letters")